In [ ]:
# GOOGLE COLLAB
_='''
! rm -rf __MACOSX/ landscapes data
! mkdir data
! curl https://s3.amazonaws.com/artemis.ai/landscapes.zip --output images.zip
! unzip images.zip
! rm -rf __MACOSX/
! mv landscapes data/landscapes
'''

In [ ]:
import torch

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

from time import time
from skimage import io, color
from torchvision import datasets
from skimage.color import lab2rgb, rgb2lab, rgb2gray

%matplotlib inline

In [ ]:
!pip install split-folders
import splitfolders
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("data/landscapes", output="output", seed=1337, ratio=(.8, .2), group_prefix=None) # default values

In [ ]:

class GrayscaleImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.imgs[index]
        # Get rgb / lab
        rgb = self.loader(path)
        rgb = self.transform(rgb)
        rgb = np.asarray(rgb)
        lab = color.rgb2lab(rgb)
        #lab = (lab + 128) / 255
        
        gray, lab = lab[:,:,0], lab[:, :, 1:3]
        
        # Transpose from WxHxC to CxWxH
        rgb = np.transpose(rgb, (2, 0, 1))
        lab = np.transpose(lab, (2, 0, 1))
        
        rgb = torch.from_numpy(rgb).float() / 255
        gray = torch.from_numpy(gray).unsqueeze(0).float()
        lab = torch.from_numpy(lab).float()
        
        return rgb, lab, gray

transform = transforms.Compose([
    #transforms.Grayscale(),
    transforms.Resize((256,256)),
    #transforms.ToTensor(),
])

dataset = GrayscaleImageFolder("./output/train", transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, num_workers=0)

dataiter = iter(data_loader)
rgb, lab, gray = dataiter.next()

fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(10,4))

axes[0].imshow(np.transpose(rgb[0], (1, 2, 0)))
axes[1].imshow(gray[0][0], cmap='gray')
axes[2].imshow(lab[0][0], cmap='gray')

In [ ]:
# convert data to torch.FloatTensor
transform = transforms.Compose([
    #transforms.Grayscale(),
    transforms.Resize((256,256)),
    transforms.ToTensor(),
])

transform_gray = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((256,256)),
    transforms.ToTensor(),
])

# load the training and test datasets
dataset = datasets.ImageFolder("./output/train", transform=transform)
dataset_gray = datasets.ImageFolder("./output/train", transform=transform_gray)

dataset_test = datasets.ImageFolder("./output/val", transform=transform)
dataset_gray_test = datasets.ImageFolder("./output/val", transform=transform_gray)
#train_data = datasets.CelebA(root='data', download=True, transform=transform)

In [ ]:
# Create training and test dataloaders
num_workers = 0
# how many samples per batch to load
batch_size = 64

# prepare data loaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)
data_gray_loader = torch.utils.data.DataLoader(dataset_gray, batch_size=batch_size, num_workers=num_workers)

data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, num_workers=num_workers)
data_gray_loader_test = torch.utils.data.DataLoader(dataset_gray_test, batch_size=batch_size, num_workers=num_workers)


In [ ]:
# Show the first image
dataiter = iter(data_loader)
images, labels = dataiter.next()
plt.imshow(np.transpose(dataset[0][0].numpy(), (1,2,0)))

In [ ]:
dataiter = iter(data_gray_loader)
images, labels = dataiter.next()
plt.imshow(dataset_gray[0][0][0].numpy(), cmap='gray')

In [ ]:

# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        
        # Batch norm
        self.conv2_bn = nn.BatchNorm2d(24)
        
        # First conv layers
        self.conv1 = nn.Conv2d(1, 32, 7, padding=3)
        #self.conv2 = nn.Conv2d(64, 32, 3, padding=1)
        #self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        # First inception block
        self.conv1_1 = nn.Conv2d(32, 8, 5, padding=2)
        self.conv1_2 = nn.Conv2d(8, 8, 3, padding=1)
        self.conv1_3 = nn.Conv2d(8, 8, 1, padding=0)
        # Second inception block
        self.conv2_1 = nn.Conv2d(24, 8, 5, padding=2)
        self.conv2_2 = nn.Conv2d(8, 8, 3, padding=1)
        self.conv2_3 = nn.Conv2d(8, 8, 1, padding=0)
        
        # Final conv layer
        self.conv4 = nn.Conv2d(24,3, 1, padding=0)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        #x = F.relu(self.conv3(x))
        # First inception block
        x1_1 = F.relu(self.conv1_1(x))
        x1_2 = F.relu(self.conv1_2(x1_1))
        x1_3 = F.relu(self.conv1_3(x1_2))
        block1 = [x1_1, x1_2, x1_3]
        x = torch.cat(block1, 1)
        x = self.conv2_bn(x)
        # Second inception block
        x2_1 = F.relu(self.conv2_1(x))
        x2_2 = F.relu(self.conv2_2(x2_1))
        x2_3 = F.relu(self.conv2_3(x2_2))
        x = torch.cat((x2_1, x2_2, x2_3), 1)
        
        x = F.sigmoid(self.conv4(x))
        return x

# initialize the NN
model = ConvAutoencoder()

# Testing purposes 
X = torch.rand((1,1,64,64))
Y = model(X)
print(X.shape, Y.shape)

In [ ]:
print(f'Using {["CPU", "CUDA"][torch.cuda.is_available()]}')
cuda_available = torch.cuda.is_available()

if cuda_available:
    model = model.cuda()
# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# number of epochs to train the model
n_epochs = 1

model.train()
for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    
    for data, data_gray in zip(data_loader, data_gray_loader):
        #start = time()
        # Get both images, images_gray is the input, images is the expected output
        images_gray, _ = data_gray
        images, _ = data

        if cuda_available:
            images_gray = images_gray.cuda()
            images = images.cuda()

        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(images_gray)
       
        # calculate the loss
        loss = criterion(outputs, images)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*images.size(0)
        
        #print(time() - start)
        #break
    # print avg training statistics 
    train_loss = train_loss/len(data_loader)
    print(
        'Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch, 
            train_loss
        )
    )

In [ ]:
n_inputs = 5
model.cpu()
dataiter = iter(data_gray_loader_test)
X, _ = dataiter.next()[:5]
Y = model(X).cpu().detach().numpy()

dataiter = iter(data_loader_test)
X, _ = dataiter.next()
fig, axes = plt.subplots(nrows=2, ncols=n_inputs, sharex=True, sharey=True, figsize=(10,4))
for i in range(n_inputs):
    x = np.transpose(X[i+26], (1, 2, 0))
    y = np.transpose(Y[i+26], (1, 2, 0))
    axes[0][i].imshow(x)
    axes[1][i].imshow(y)